In [104]:
#import required packages
import requests
import json
import os
import sqlite3 
import pandas as pd
from decouple import config
from geopy.geocoders import Nominatim

In [7]:
df = pd.read_csv("houses_cleaned_baseline_crimedata.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,final_price,bathrooms,parking,description,type,full_link,full_address,lat,...,OBJECTID,Neighbourhood,Hood_ID,Shape__Area,Shape__Length,Assault_Rate,Auto_Theft_Rate,Break_andEnter_Rate,Robbery_Rate,Shootings_Rate
0,0,0,855000,2,1,"Luxurious And Spacious Murano Tower. 2+1, 2 Ba...",Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"38 Grenville St, Toronto , Ontario, Canada",43.661896,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
1,1,85,785000,1,1,Five Condos - Stunning Bright Corner Unit In A...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"5 St Joseph St, Toronto , Ontario, Canada",43.665753,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
2,2,154,718000,2,0,"Spacious 2 Bedrooms, 2 Full Baths, Over 1000 S...",Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"633 Bay St, Toronto , Ontario, Canada",43.656674,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
3,3,161,968000,2,1,Beautiful Burano On Bay Is A Diamond In The Sk...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"832 Bay St, Toronto , Ontario, Canada",43.661927,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211
4,4,216,650000,2,1,*Smart & Functional One Bdrm + Den + Extra Bat...,Condo Apt,https://www.zoocasa.com/toronto-on-real-estate...,"111 Elizabeth St, Toronto , Ontario, Canada",43.655055,...,94,Bay Street Corridor,76,1.809355e+06,6933.183322,2511.6408,100.95383,532.97306,371.57816,12.331211


In [86]:
neighbourhoods = df.Neighbourhood.unique()
neighbourhoods

array(['Bay Street Corridor', 'Islington-City Centre West',
       'Mount Pleasant West', 'Niagara', 'New Toronto',
       'Lawrence Park South', 'Glenfield-Jane Heights',
       'Mount Pleasant East', 'Malvern', 'Junction Area',
       'Wexford/Maryvale', 'Cabbagetown-South St.James Town',
       'St.Andrew-Windfields', 'Banbury-Don Mills', 'Milliken',
       'Mimico (includes Humber Bay Shores)', 'West Hill',
       'Birchcliffe-Cliffside', 'Dorset Park', 'Yorkdale-Glen Park',
       'Church-Yonge Corridor', 'Rockcliffe-Smythe', "O'Connor-Parkview",
       'York University Heights', 'South Parkdale',
       'Downsview-Roding-CFB', 'Trinity-Bellwoods',
       'Dovercourt-Wallace Emerson-Junction', 'Willowdale West',
       'Greenwood-Coxwell', 'Markland Wood', 'Scarborough Village',
       'Victoria Village', 'East End-Danforth', 'North St.James Town',
       'Ionview', 'Rustic', 'Waterfront Communities-The Island',
       'Mount Dennis', 'Oakwood Village', 'Annex', 'Flemingdon Park',

In [126]:
#get coordinates of every neighbourhood

hoods = {}
hood_nan = []
for neighbourhood in neighbourhoods:
    try:
        address = neighbourhood + ', ' + 'Toronto, Canada'
        location = geolocator.geocode(address)
        lat = location.latitude
        lng = location.longitude
        hoods[neighbourhood] = str(lat)+","+ str(lng)
        
    except:
        print (neighbourhood)
        hood_nan.append(neighbourhood)

Lawrence Park South
Glenfield-Jane Heights
Junction Area
Wexford/Maryvale
Cabbagetown-South St.James Town
St.Andrew-Windfields
Mimico (includes Humber Bay Shores)
Birchcliffe-Cliffside
O'Connor-Parkview
Downsview-Roding-CFB
Greenwood-Coxwell
Waterfront Communities-The Island
Agincourt South-Malvern West
Bedford Park-Nortown
Newtonbrook East
Humbermede
Mount Olive-Silverstone-Jamestown
Princess-Rosethorn
Briar Hill-Belgravia
Kingsview Village-The Westway
Elms-Old Rexdale
Bridle Path-Sunnybrook-York Mills
Clairlea-Birchmount
Woodbine Corridor
Newtonbrook West
Woodbine-Lumsden
Pelmo Park-Humberlea
Parkwoods-Donalda
Lansing-Westgate
Forest Hill South
Westminster-Branson
Brookhaven-Amesbury
Eringate-Centennial-West Deane
Blake-Jones
Thistletown-Beaumond Heights
Weston-Pellam Park
Willowridge-Martingrove-Richview
Beechborough-Greenbrook
Caledonia-Fairbank


In [132]:
hood_nan

['Lawrence Park South',
 'Glenfield-Jane Heights',
 'Junction Area',
 'Wexford/Maryvale',
 'Cabbagetown-South St.James Town',
 'St.Andrew-Windfields',
 'Mimico (includes Humber Bay Shores)',
 'Birchcliffe-Cliffside',
 "O'Connor-Parkview",
 'Downsview-Roding-CFB',
 'Greenwood-Coxwell',
 'Waterfront Communities-The Island',
 'Agincourt South-Malvern West',
 'Bedford Park-Nortown',
 'Newtonbrook East',
 'Humbermede',
 'Mount Olive-Silverstone-Jamestown',
 'Princess-Rosethorn',
 'Briar Hill-Belgravia',
 'Kingsview Village-The Westway',
 'Elms-Old Rexdale',
 'Bridle Path-Sunnybrook-York Mills',
 'Clairlea-Birchmount',
 'Woodbine Corridor',
 'Newtonbrook West',
 'Woodbine-Lumsden',
 'Pelmo Park-Humberlea',
 'Parkwoods-Donalda',
 'Lansing-Westgate',
 'Forest Hill South',
 'Westminster-Branson',
 'Brookhaven-Amesbury',
 'Eringate-Centennial-West Deane',
 'Blake-Jones',
 'Thistletown-Beaumond Heights',
 'Weston-Pellam Park',
 'Willowridge-Martingrove-Richview',
 'Beechborough-Greenbrook',
 'Cal

In [135]:
#Manually adding coordinates for each missing neighbourhood

hoods['Lawrence Park South'] = str(43.7121)+","+ str(-79.4117)
hoods['Glenfield-Jane Heights'] = str(43.74675)+","+ str(-79.51504)
hoods['Junction Area'] = str(43.6655)+","+ str(-79.4721)
hoods['Wexford/Maryvale'] = str(43.7613)+","+ str(-79.3008)
hoods['Cabbagetown-South St.James Town'] = str(43.6664)+","+ str(-79.3629)
hoods['St.Andrew-Windfields'] = str(43.7548)+","+ str(-79.3855)
hoods['Mimico (includes Humber Bay Shores)'] = str(43.6146)+","+ str(-79.4957)
hoods['Birchcliffe-Cliffside'] = str(43.6875)+","+ str(-79.2694)
hoods["O'Connor-Parkview"] = str(43.7061)+","+ str(-79.3102)
hoods['Downsview-Roding-CFB'] = str(43.73329)+","+ str( -79.49049)
hoods['Greenwood-Coxwell'] = str(43.6721)+","+ str(-79.3228)
hoods['Waterfront Communities-The Island'] = str(43.644)+","+ str(-79.380218)
hoods['Agincourt South-Malvern West'] = str(43.789)+","+ str(-79.26941)
hoods['Bedford Park-Nortown'] = str(43.7303)+","+ str(-79.4114)
hoods['Newtonbrook East'] = str(43.78333)+","+ str(-79.41666)
hoods['Humbermede'] = str(43.7390)+","+ str(-79.5394)
hoods['Mount Olive-Silverstone-Jamestown'] = str(43.74687)+","+ str(-79.58726)
hoods['Princess-Rosethorn'] = str(43.6700)+","+ str(-79.5477)
hoods['Briar Hill-Belgravia'] = str(43.7037)+","+ str(-79.4524)
hoods['Kingsview Village-The Westway'] = str(43.69944)+","+ str(-79.54924)
hoods['Elms-Old Rexdale'] = str(43.72429)+","+ str(-79.54956)
hoods['Bridle Path-Sunnybrook-York Mills'] = str(43.7359)+","+ str(-79.3719)
hoods['Clairlea-Birchmount'] = str(43.7185)+","+ str(-79.2914)
hoods['Woodbine Corridor'] = str(43.682088)+","+ str(-79.3032772)
hoods['Newtonbrook West'] = str(43.7871)+","+ str(-79.4173)
hoods['Woodbine-Lumsden'] = str(43.6922)+","+ str(-79.3099)
hoods['Pelmo Park-Humberlea'] = str(43.7104)+","+ str(-79.5219)
hoods['Parkwoods-Donalda'] = str(43.7607)+","+ str(-79.3277)
hoods['Lansing-Westgate'] = str(43.7590)+","+ str(-79.4226)
hoods['Forest Hill South'] = str(43.6932)+","+ str(-79.4126)
hoods['Westminster-Branson'] = str(43.7856)+","+ str(-79.4511)
hoods['Brookhaven-Amesbury'] = str(43.6969)+","+ str(-79.4938)
hoods['Eringate-Centennial-West Deane'] = str(43.6599)+","+ str(-79.5833)
hoods['Blake-Jones'] = str(43.6759)+","+ str(-79.3381)
hoods['Thistletown-Beaumond Heights'] = str(43.7372)+","+ str(-79.5653)
hoods['Weston-Pellam Park'] = str(43.6716)+","+ str(-79.4577)
hoods['Willowridge-Martingrove-Richview'] = str(43.6789276752)+","+ str(-79.5715452921)
hoods['Beechborough-Greenbrook'] = str(43.6943)+","+ str(-79.4748)
hoods['Caledonia-Fairbank'] = str(43.6899)+","+ str(-79.4552)

### Foursquare API

In [42]:
fsquare_clientid = str(config('fsquare_clientid'))
fsquare_key = str(config('fsquare_key'))
fsquare_clientsecret = str(config('fsquare_clientsecret'))
version = '20180605'
limit = 100000

In [79]:
def toronto_venues(names, latitudes, longitudes, radius=500):
    
    headers = {"Accept": "application/json"}
    headers['Authorization'] = fsquare_key

    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        coordin = str(lat)+","+str(lng)
        print (coordin)
        # create the API request URL
        url = 'https://api.foursquare.com/v3/places/search?&ll={}&radius={}&limit={}'.format(
            fsquare_clientid, fsquare_clientsecret, str(lat)+","+str(lng), radius, limit)

        # make the GET request
        results = requests.get(url).json()

#         results = requests.get(url).json()["response"]['groups'][0]['items']

#         # return only relevant information for each nearby venue
#         venues_list.append([(name, lat, lng, v['venue']['name'],
#                              v['venue']['categories'][0]['name'])
#                             for v in results])

#     venues = pd.DataFrame(
#         [item for venue_list in venues_list for item in venue_list])
#     venues.columns = ['Neighbourhood Name', 'Neighbourhood Latitude',
#         'Neighbourhood Longitude', 'Venue', 'Venue Category']

#     return (venues)
    return (results)

In [80]:
#  Get Venues for each Neighbourhood
toronto_venues = toronto_venues(names = df['Neighbourhood'].head(),
                                 latitudes = df['lat'].head(),
                                 longitudes = df['long'].head())

43.6618956,-79.3857479
43.6657528,-79.3855792
43.656674,-79.3834432
43.661927,-79.3865112
43.6550554,-79.3847211


In [78]:
toronto_venues

{'message': 'Invalid request token.'}

In [97]:
toronto_venues =[]

for i in neighbourhoods:
    url = "https://api.foursquare.com/v3/places/search"

    headers = {"Accept": "application/json"}
    headers['Authorization'] = fsquare_key

    params = {'near':i,'radius':'500','categories':'10000,11000,12000,13000,14000,15000,16000,17000,18000,19000','limit':100}

    response = requests.get(url, headers=headers, params = params)
    toronto_venues.append(response.json())


In [120]:
toronto_venues =[]
url = "https://api.foursquare.com/v3/places/search"

headers = {"Accept": "application/json"}
headers['Authorization'] = fsquare_key

params = {'near':'Humbermede','categories':'10000,11000,12000,13000,14000,15000,16000,17000,18000,19000', 'limit':50}

response = requests.get(url, headers=headers, params = params)

toronto_venues.append(response.json())

toronto_venues_json = (response.json()).copy()

In [121]:
toronto_venues

[{'results': [{'fsq_id': '4c0aa69fa1b32d7fceb799f0',
    'categories': [{'id': 13379,
      'name': 'Vietnamese Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vietnamese_',
       'suffix': '.png'}}],
    'chains': [],
    'distance': 988,
    'geocodes': {'main': {'latitude': 43.749843, 'longitude': -79.549745},
     'roof': {'latitude': 43.749843, 'longitude': -79.549745}},
    'link': '/v3/places/4c0aa69fa1b32d7fceb799f0',
    'location': {'address': '2437 Finch Ave W',
     'address_extended': '# 4',
     'country': 'CA',
     'cross_street': '',
     'formatted_address': '2437 Finch Ave W, Toronto ON M9M 2E7',
     'locality': 'Toronto',
     'neighborhood': ['Humbermede'],
     'postcode': 'M9M 2E7',
     'region': 'ON'},
    'name': 'Pho Dau Bo Restaurant',
    'related_places': {},
    'timezone': 'America/Toronto'},
   {'fsq_id': '4e1dc152e4cd450a85742f7f',
    'categories': [{'id': 13039,
      'name': 'Deli',
      'icon': {'prefix': 'http

In [118]:
toronto_venues_df = pd.json_normalize(toronto_venues_json['results'], max_level = 1)

In [119]:
toronto_venues_df

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main,geocodes.roof,location.address,...,location.country,location.cross_street,location.dma,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.po_box,related_places.children
0,4f70ac2ee4b0ac4cdd2bdbd1,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[],1800,/v3/places/4f70ac2ee4b0ac4cdd2bdbd1,City Brew,America/Denver,"{'latitude': 45.811259, 'longitude': -108.472202}","{'latitude': 45.811259, 'longitude': -108.472202}",405 Main St,...,US,,Billings,"405 Main St, Billings, MT 59105",Billings,[Billings Heights],59105,MT,NaN,NaN
1,4bba705c1261d13affa5ea98,"[{'id': 13026, 'name': 'BBQ Joint', 'icon': {'...",[],1606,/v3/places/4bba705c1261d13affa5ea98,Blues BBQ,America/Denver,"{'latitude': 45.813149, 'longitude': -108.472935}","{'latitude': 45.813149, 'longitude': -108.472935}",523 Hilltop Rd,...,US,btwn Montgomery & Radford Square S.,Billings,523 Hilltop Rd (btwn Montgomery & Radford Squa...,Billings,[Billings Heights],59105,MT,NaN,NaN
2,56e84612498eb866a942dfbd,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da'...,2152,/v3/places/56e84612498eb866a942dfbd,Starbucks,America/Denver,"{'latitude': 45.806489, 'longitude': -108.475548}","{'latitude': 45.806489, 'longitude': -108.475548}",406 Main St,...,US,,Billings,"406 Main St, Billings, MT 59105",Billings,NaN,59105,MT,NaN,NaN
3,4ba51fcaf964a5208be038e3,"[{'id': 13030, 'name': 'Buffet', 'icon': {'pre...",[],1750,/v3/places/4ba51fcaf964a5208be038e3,Montana Jacks-Billings,America/Denver,"{'latitude': 45.811604, 'longitude': -108.473313}","{'latitude': 45.811604, 'longitude': -108.473313}",520 Hansen Ln,...,US,Main st.,Billings,"520 Hansen Ln (Main st.), Billings, MT 59105",Billings,[Billings Heights],59105,MT,NaN,NaN
4,4c4650993f0276b067f152e7,"[{'id': 17090, 'name': 'Hardware Store', 'icon...",[{'id': 'ab4c5650-d68a-012e-5619-003048cad9da'...,1315,/v3/places/4c4650993f0276b067f152e7,Ace Hardware,America/Denver,"{'latitude': 45.82689, 'longitude': -108.469302}","{'latitude': 45.82689, 'longitude': -108.469302}",1547 Main St,...,US,,Billings,"1547 Main St, Billings, MT 59105",Billings,[Billings Heights],59105,MT,P.O. Box 50189,NaN
5,4bddfd6ee75c0f478a72c603,"[{'id': 13068, 'name': 'American Restaurant', ...",[],2459,/v3/places/4bddfd6ee75c0f478a72c603,Bull Mountain Grille,America/Denver,"{'latitude': 45.841546, 'longitude': -108.469176}","{'latitude': 45.841546, 'longitude': -108.469176}",2376 Main St,...,US,btwn Pemberton & Roundup,Billings,"2376 Main St (btwn Pemberton & Roundup), Billi...",Billings,NaN,59105,MT,NaN,NaN
6,4bfdb9198992a593cd95acb0,"[{'id': 13064, 'name': 'Pizzeria', 'icon': {'p...",[],1590,/v3/places/4bfdb9198992a593cd95acb0,Godfathers Pizza,America/Denver,"{'latitude': 45.815195, 'longitude': -108.470313}","{'latitude': 45.815195, 'longitude': -108.470313}",905 Main St,...,US,,Billings,"905 Main St, Billings, MT 59105",Billings,[Billings Heights],59105,MT,NaN,NaN
7,4c54423d72cf0f4716853dd4,"[{'id': 17035, 'name': 'Drugstore', 'icon': {'...",[{'id': 'ab4b4810-d68a-012e-5619-003048cad9da'...,1145,/v3/places/4c54423d72cf0f4716853dd4,Walgreens,America/Denver,"{'latitude': 45.827957, 'longitude': -108.471818}","{'latitude': 45.827957, 'longitude': -108.471818}",1602 Main St,...,US,,Billings,"1602 Main St, Billings, MT 59105",Billings,[Billings Heights],59105,MT,NaN,NaN
8,4cefae57b80da093d5a62d00,"[{'id': 17000, 'name': 'Retail', 'icon': {'pre...",[{'id': 'fadd10d6-a182-4428-a3d8-849932901b69'...,2043,/v3/places/4cefae57b80da093d5a62d00,Tractor Supply Company,America/Denver,"{'latitude': 45.807873, 'longitude': -108.474939}","{'latitude': 45.807873, 'longitude': -108.474939}",496 Main St,...,US,,Billings,"496 Main St, Billings, MT 59105",Billings,NaN,59105,MT,NaN,NaN
9,4bf2dd1c94af2d7ff0d63872,"[{'id': 12075, 'name': 'Post Office', 'icon': ...",[{'id': '30c40284-04fb-4ebb-8406-f3ac956bd976'...,1415,/v3/pla